In [1]:
from openai import OpenAI
import os
import numpy as np
import pandas as pd
import json
DB_PATH = '../data'

In [2]:
with open('api_key.txt','r') as f:
    apikey = f.read()
client = OpenAI(api_key=apikey)

In [3]:
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return np.array(client.embeddings.create(input = [text], model=model).data[0].embedding)


In [4]:
query = 'who was the mother of Afonso II, the third king of protugal'

response = client.chat.completions.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions regarding the input.'},
        {'role': 'user', 'content': query},
    ],
    model='gpt-4',
    temperature=0,
)

response_content = response.choices[0].message.content

In [5]:
hall_answer = "The mother of Afonso II, the third king of Portugal, was Urraca of Castile. Urraca was a Spanish infanta and the wife of King Sancho I of Portugal, making her the queen consort of Portugal and the mother of Afonso II."

In [6]:
print(query)
print(response_content)
print(hall_answer)

who was the mother of Afonso II, the third king of protugal
The mother of Afonso II, the third king of Portugal, was Queen Dulce of Aragon.
The mother of Afonso II, the third king of Portugal, was Urraca of Castile. Urraca was a Spanish infanta and the wife of King Sancho I of Portugal, making her the queen consort of Portugal and the mother of Afonso II.


In [13]:
q_emb = get_embedding(query)
true_emb = get_embedding(response_content)
false_emb = get_embedding(hall_answer)

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
cos_q_t = cosine_similarity(q_emb.reshape(1, -1),true_emb.reshape(1, -1))
cos_q_f = cosine_similarity(q_emb.reshape(1, -1),false_emb.reshape(1, -1))
cos_t_f = cosine_similarity(true_emb.reshape(1, -1),false_emb.reshape(1, -1))
print(f'''
q and true similarity is: {cos_q_t},
q and false similarity is: {cos_q_f},
true and false similarity is: {cos_t_f},
''')


q and true similarity is: [[0.57505038]],
q and false similarity is: [[0.51143003]],
true and false similarity is: [[0.50097567]],


# Factor DF

In [71]:
expert_factor_df = pd.read_csv(os.path.join(DB_PATH,'Factor/expert_factor.csv'),delimiter=',')
news_factor_df = pd.read_csv(os.path.join(DB_PATH,'Factor/news_factor.csv'),delimiter=',')
wiki_factor_df = pd.read_csv(os.path.join(DB_PATH,'Factor/wiki_factor.csv'),delimiter=',')

In [72]:
n = 1
expert_factor_df.iloc[n]

Unnamed: 0.1                                                           1
Unnamed: 0                                                             1
full_prefix            Question: How long does it take to become a re...
context                Question: How long does it take to become a re...
doc_id                                                                 6
completion             Becoming a real estate agent generally takes a...
contradiction_0        Becoming a real estate agent generally takes a...
contradiction_1        Becoming a real estate agent generally takes a...
contradiction_2        Becoming a real estate agent generally takes a...
longest_completions                                                 27.0
turncated_prefixes     Question: How long does it take to become a re...
Name: 1, dtype: object

In [73]:
q = expert_factor_df.iloc[n].context

In [74]:
t = expert_factor_df.iloc[n].completion

In [75]:
f1 = expert_factor_df.iloc[n].contradiction_0

In [76]:
f2 = expert_factor_df.iloc[n].contradiction_1

In [77]:
f3 = expert_factor_df.iloc[n].contradiction_2

In [78]:
q_emb = get_embedding(q).reshape(1, -1)
t_emb = get_embedding(t).reshape(1, -1)
f1_emb = get_embedding(f1).reshape(1, -1)
f2_emb = get_embedding(f2).reshape(1, -1)
f3_emb = get_embedding(f3).reshape(1, -1)

In [79]:
cos_q_t = cosine_similarity(q_emb,t_emb)
cos_q_f = cosine_similarity(q_emb,np.vstack([f1_emb,f2_emb,f3_emb]))
cos_t_f = cosine_similarity(t_emb,np.vstack([f1_emb,f2_emb,f3_emb]))
print(f'''
q and t similarity is: {cos_q_t},
q and false similarity is: {cos_q_f},
true and false similarity is: {cos_t_f},
''')


q and t similarity is: [[0.72715646]],
q and false similarity is: [[0.73391262 0.72647    0.71169414]],
true and false similarity is: [[0.86439275 0.90650554 0.89166153]],


In [81]:
t

'Becoming a real estate agent generally takes about 1-3 months according to the laws in the state where the student wishes to work .'

In [82]:
f2

'Becoming a real estate agent generally takes at least one year according to the laws in the state where the student wishes to work.'

In [83]:
f1

'Becoming a real estate agent generally takes about 2-4 years according to the laws in the state where the student wishes to work.'

In [84]:
f3

'Becoming a real estate agent generally takes a few weeks according to the laws in the state where the student wishes to work.'

# HaluEval

In [98]:
def load_qa(filename):
    text = []
    right_answers = []
    hall_answers = []
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            sample = json.loads(line)
            if "knowledge" in sample:
                text.append(sample["knowledge"] +" "+ sample["question"])
                right_answers.append(sample['right_answer'])
                hall_answers.append(sample['hallucinated_answer'])
    return text, right_answers,hall_answers

In [99]:
qa_fp = os.path.join(DB_PATH,'HaluEval/qa_data.json')
qa_save_fp = os.path.join(DB_PATH,'HaluEval/qa_emb_q_t_f.npy')

In [100]:
def save_emb(fp,save_fp):
    q_embs = []
    t_embs = []
    f_embs = []
    text, right_answers,hall_answers = load_qa(fp)
    for q,t,f in zip(text, right_answers,hall_answers):
        q_embs.append(get_embedding(q))
        t_embs.append(get_embedding(t))
        f_embs.append(get_embedding(f))
    q_emb_arr = np.vstack(q_embs)
    t_emb_arr = np.vstack(t_embs)
    f_emb_arr = np.vstack(f_embs)
    
    overall_emb = np.stack((q_emb_arr, t_emb_arr, f_emb_arr), axis=0)
    np.save(save_fp,overall_emb)
    print(f'Saved at {save_fp}')

In [ ]:
save_emb(qa_fp,qa_save_fp)

In [ ]:
emb = np.load(qa_save_fp)
emb.shape

In [53]:
a = df_test.q_emb.values

In [69]:
text, right_answers,hall_answers = load_qa(qa_fp)

In [90]:
a.shape

(3, 1, 1536)

AttributeError: 'list' object has no attribute 'replace'